# Agentic Retrieval Quickstart for Azure AI Search

### 1. Load Connections

In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
answer_model = os.getenv("ANSWER_MODEL", "gpt-4o")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4o")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4o")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-03-01-preview")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent")
api_version = "2025-05-01-Preview"

### 2. Create Index

In [2]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient

index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")


Index 'earth_at_night' created or updated successfully


### 3. Upload Sample Documents

In [3]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}'")


Documents uploaded to index 'earth_at_night'


### 4. Create search agent

In [4]:
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeAgentTargetIndex, KnowledgeAgentRequestLimits, AzureOpenAIVectorizerParameters

agent = KnowledgeAgent(
    name=agent_name,
    models=[
        KnowledgeAgentAzureOpenAIModel(
            azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint,
                deployment_name=azure_openai_gpt_deployment,
                model_name=azure_openai_gpt_model
            )
        )
    ],
    target_indexes=[
        KnowledgeAgentTargetIndex(
            index_name=index_name,
            default_reranker_threshold=2.5
        )
    ],
)

index_client.create_or_update_agent(agent)
print(f"Knowledge agent '{agent_name}' created or updated successfully")


Knowledge agent 'earth-search-agent' created or updated successfully


### 5. Setup messages

In [5]:
instructions = """
An Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

### 6. Use Agentic Retrieval to fetch results

In [6]:
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams
import azure.search.documents._utils

agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=agent_name, authentication_policy=azure.search.documents._utils.get_authentication_policy(credential))

messages.append({
    "role": "user",
    "content": """
    Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
    Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
})

retrieval_result = agent_client.knowledge_retrieval.retrieve(
    retrieval_request=KnowledgeAgentRetrievalRequest(
        messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg["content"])]) for msg in messages],
        target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2.5)]
    )
)
messages.append({
    "role": "assistant",
    "content": retrieval_result.response[0].content[0].text
})

### 6.1. Review retrieval activity and results

In [7]:
import textwrap

print("Response")
print(textwrap.fill(retrieval_result.response[0].content[0].text, width=120))

Response
[{"ref_id":1,"content":"# Urban Structure\n\n## March 16, 2013\n\n### Phoenix Metropolitan Area at Night\n\nThis figure
presents a nighttime satellite view of the Phoenix metropolitan area, highlighting urban structure and transport
corridors. City lights illuminate the layout of several cities and major thoroughfares.\n\n**Labeled Urban
Features:**\n\n- **Phoenix:** Central and brightest area in the right-center of the image.\n- **Glendale:** Located to
the west of Phoenix, this city is also brightly lit.\n- **Peoria:** Further northwest, this area is labeled and its
illuminated grid is seen.\n- **Grand Avenue:** Clearly visible as a diagonal, brightly lit thoroughfare running from
Phoenix through Glendale and Peoria.\n- **Salt River Channel:** Identified in the southeast portion, running through
illuminated sections.\n- **Phoenix Mountains:** Dark, undeveloped region to the northeast of Phoenix.\n- **Agricultural
Fields:** Southwestern corner of the image, grid patterns are 

In [8]:
import json
print("Activity")
print(json.dumps([a.as_dict() for a in retrieval_result.activity], indent=2))
print("Results")
print(json.dumps([r.as_dict() for r in retrieval_result.references], indent=2))

Activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1407,
    "output_tokens": 318
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "suburban belts December brightening urban cores light levels"
    },
    "query_time": "2025-05-02T04:50:56.846Z",
    "elapsed_ms": 611
  },
  {
    "id": 2,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "Phoenix nighttime street grid visibility from space"
    },
    "query_time": "2025-05-02T04:50:57.331Z",
    "count": 2,
    "elapsed_ms": 485
  }
]
Results
[
  {
    "type": "AzureSearchDoc",
    "id": "0",
    "activity_source": 2,
    "doc_key": "earth_at_night_508_page_104_verbalized"
  },
  {
    "type": "AzureSearchDoc",
    "id": "1",
    "activity_source": 2,
    "doc_key": "earth_at_night_508_page_105_verbalized"
  }
]


### 7. Create Azure OpenAI Client

In [9]:
from openai import AzureOpenAI
from azure.identity import get_bearer_token_provider

azure_openai_token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    azure_ad_token_provider=azure_openai_token_provider,
    api_version=azure_openai_api_version
)

### 7.1 Use Responses API to generate an answer

In [10]:
response = client.responses.create(
    model=answer_model,
    input=messages
)

wrapped = textwrap.fill(response.output_text, width=100)
print(wrapped)

Suburban belts display larger December brightening than urban cores because seasonal lighting, such
as holiday decorations, tends to be more prevalent in residential areas where individual homes often
contribute significantly to increased brightness. In contrast, urban cores, while having higher
absolute light levels, don't experience as much variability due to the consistent presence of
streetlights, commercial lighting, and less individual contribution to seasonal displays [ref_id:0].
The Phoenix nighttime street grid is sharply visible from space because it is laid out in a regular
grid of city blocks and streets, typical of western U.S. cities. This lighting pattern is evident
from the low-Earth-orbit vantage point and highlights the extensive spread of urbanization. In
contrast, large stretches of interstate between midwestern cities remain comparatively dim as these
areas primarily consist of long, undeveloped stretches without urban lighting patterns, typical of
agricultural or 

### 7.2 Use Chat Completions API to generate an answer

In [11]:
response = client.chat.completions.create(
    model=answer_model,
    messages=messages
)

wrapped = textwrap.fill(response.choices[0].message.content, width=100)
print(wrapped)

Suburban belts display larger December brightening compared to urban cores even though absolute
light levels are higher downtown due to the widespread use of decorative lighting during the holiday
season. Suburban areas tend to have more residential properties where festive lighting is commonly
used, contributing to this phenomenon. Additionally, suburban development often spreads outward,
leading to a more extensive area where lighting may be present compared to the concentrated urban
cores {ref_id: 0}.  Regarding Phoenix's sharply visible nighttime street grid from space, Phoenix
and many cities in the western United States feature a regular grid layout of city blocks and
streets. This grid is particularly evident at night because street lighting creates a clear and
sharp pattern visible from space. In contrast, the interstate between midwestern cities may appear
dimmer because these areas generally have less consistent lighting and lower population density,
causing stretches of road

### 8. Continue the conversation

In [12]:
messages.append({
    "role": "user",
    "content": "How do I find lava at night?"
})

retrieval_result = agent_client.knowledge_retrieval.retrieve(
    retrieval_request=KnowledgeAgentRetrievalRequest(
        messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg["content"])]) for msg in messages],
        target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2.5)]
    )
)
messages.append({
    "role": "assistant",
    "content": retrieval_result.response[0].content[0].text
})

### 8.1. Review activity and results

In [13]:
print("Response")
print(textwrap.fill(retrieval_result.response[0].content[0].text, width=120))

Response
[{"ref_id":5,"content":"## Nature's Light Shows\n\nAt night, with the light of the Sun removed, nature's brilliant glow
from Earth's surface becomes visible to the naked eye from space. Some of Earth's most spectacular light shows are
natural, like the aurora borealis, or Northern Lights, in the Northern Hemisphere (aurora australis, or Southern Lights,
in the Southern Hemisphere). The auroras are natural electrical phenomena caused by charged particles that race from the
Sun toward Earth, inducing chemical reactions in the upper atmosphere and creating the appearance of streamers of
reddish or greenish light in the sky, usually near the northern or southern magnetic pole. Other natural lights can
indicate danger, like a raging forest fire encroaching on a city, town, or community, or lava spewing from an erupting
volcano.\n\nWhatever the source, the ability of humans to monitor nature's light shows at night has practical
applications for society. For example, tracking fires d

In [14]:
import json
print("Activity")
print(json.dumps([a.as_dict() for a in retrieval_result.activity], indent=2))
print("Results")
print(json.dumps([r.as_dict() for r in retrieval_result.references], indent=2))

Activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 2283,
    "output_tokens": 110
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "how to locate lava flows at night"
    },
    "query_time": "2025-05-02T04:51:05.929Z",
    "count": 6,
    "elapsed_ms": 346
  }
]
Results
[
  {
    "type": "AzureSearchDoc",
    "id": "0",
    "activity_source": 1,
    "doc_key": "earth_at_night_508_page_60_verbalized"
  },
  {
    "type": "AzureSearchDoc",
    "id": "1",
    "activity_source": 1,
    "doc_key": "earth_at_night_508_page_64_verbalized"
  },
  {
    "type": "AzureSearchDoc",
    "id": "2",
    "activity_source": 1,
    "doc_key": "earth_at_night_508_page_46_verbalized"
  },
  {
    "type": "AzureSearchDoc",
    "id": "3",
    "activity_source": 1,
    "doc_key": "earth_at_night_508_page_66_verbalized"
  },
  {
    "type": "AzureSearchDoc",
    "id": "4",
    "activity_source": 1,
    "d

### 8.2. Generate answer

In [ ]:
response = client.responses.create(
    model=answer_model,
    input=messages
)

wrapped = textwrap.fill(response.output_text, width=100)
print(wrapped)